In [ ]:
import matplotlib.pyplot as plt
import pystac_client

from mccn.extent import GeoBoxBuilder
from mccn.loader import stac_load_raster

ENDPOINT = "http://203.101.230.81:8082/"
client = pystac_client.Client.open(ENDPOINT)
collection = client.get_collection("Raster")

In [ ]:
gbox = GeoBoxBuilder.from_collection(collection, 100)
items = list(collection.get_all_items())
items

In [ ]:
ds = stac_load_raster(items, gbox)
ds

In [ ]:
plt.imshow(ds.price.values[0, :, :])
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(ds.price_usd.values[0, :, :])
plt.colorbar()
plt.show()

### Load with alias

In [ ]:
alias_ds = stac_load_raster(items, gbox, band_renaming={"price_usd": "priceUSD"})
alias_ds

In [ ]:
plt.imshow(alias_ds.price.values[0, :, :])
plt.colorbar()
plt.show()

In [ ]:
plt.imshow(alias_ds.priceUSD.values[0, :, :])
plt.colorbar()
plt.show()

#### With field processing

In [ ]:
proc_ds = stac_load_raster(
    items,
    gbox,
    band_renaming={"price_usd": "price_aud"},
    band_preprocessing={"price_usd": lambda x: x / 0.67},
)
proc_ds

In [ ]:
plt.imshow(proc_ds.price_aud.values[0, :, :])
plt.colorbar()
plt.show()

In [3]:
import pystac 
import mccn.parser as Parser
from pathlib import Path 

In [4]:
path = Path("tests/files/unit_tests/point/no_date_no_alt/stac")

In [5]:
collection = pystac.Collection.from_file(path/"collection.json")

items = [] 
for item in collection.get_all_items():
    items.append(Parser.parse_item(item))

In [6]:
from mccn.loader.point import PointLoader

In [49]:
from mccn.loader.base import FilterConfig
from mccn.extent import GeoBoxBuilder
from mccn.loader.point import PointLoadConfig


filter_config = FilterConfig(geobox=GeoBoxBuilder.from_collection(collection, 120, "default"))
loader = PointLoader(items, filter_config)
ds = loader.load()

In [50]:
filter_config.geobox.coords["latitude"][0].min(), filter_config.geobox.coords["latitude"][0].max()

(np.float64(-37.92912187498107), np.float64(-37.92431229164774))

In [51]:
pos = [(-37.925731,144.644049),
(-37.929147,144.642786),
(-37.924468,144.640224),
(-37.924297,144.647704)]

for lat, lon in pos: 
    slice = ds.sel(lat=lat,lon=lon, method="nearest")
    print(lat, lon, slice.temperature.values, slice.humidity.values)

-37.925731 144.644049 [22.44] [0.61]
-37.929147 144.642786 [22.12] [0.58]
-37.924468 144.640224 [22.17] [0.55]
-37.924297 144.647704 [22.31] [0.97]


In [63]:
ds.spatial_ref

<xarray.DataArray 'spatial_ref' ()> Size: 4B
array(4326, dtype=int32)
Coordinates:
    spatial_ref  int32 4B 4326
Attributes:
    spatial_ref:                  GEOGCRS["WGS 84",ENSEMBLE["World Geodetic S...
    crs_wkt:                      GEOGCRS["WGS 84",ENSEMBLE["World Geodetic S...
    semi_major_axis:              6378137.0
    semi_minor_axis:              6356752.314245179
    inverse_flattening:           298.257223563
    reference_ellipsoid_name:     WGS 84
    longitude_of_prime_meridian:  0.0
    prime_meridian_name:          Greenwich
    geographic_crs_name:          WGS 84
    horizontal_datum_name:        World Geodetic System 1984 ensemble
    grid_mapping_name:            latitude_longitude
    GeoTransform:                 144.640199000294927600407391 0.000062333333...